# 1) Installing dependencies:-

In [ ]:
%pip install -q google-genai chromadb python-dotenv

# 2) Import and config:-

In [ ]:
from google import genai
from google.genai import types
import chromadb
import numpy as np
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = 'YOUR_API_KEY_HERE'
client = genai.Client(api_key=GOOGLE_API_KEY)

# 3) Adding Your documents:- 

In [ ]:
documents = [
    "Python is a high-level programming language known for its simplicity.",
    "JavaScript is primarily used for web development and interactive UIs.",
    "Machine learning enables computers to learn from data without explicit programming.",
    "React is a JavaScript library for building user interfaces.",
    "Data science combines statistics, programming, and domain expertise."
]

# 4) Embedding:-
         > converts text to something mathematical that we can do math on and, vectors

In [ ]:
def create_embedding_function(api_key, model_name="gemini-embedding-004"):
    client = genai.Client(api_key=api_key)
    
    def embedding_function(texts):
        if isinstance(texts, str):
            texts = [texts]
        
        response = client.models.embed_content(
            model=model_name,
            contents=texts
        )
        
        return [embedding.values for embedding in response.embeddings]
    
    return embedding_function

client = chromadb.Client()
embedding_fn = create_embedding_function(
    api_key=GOOGLE_API_KEY,
)
collection = client.get_or_create_collection(
    name="documents",
    embedding_function=embedding_fn
)

for i, doc in enumerate(documents):
    collection.add(
        documents=[doc],
        ids=[f"doc_{i}"]
    )

# 5) Querying:-
            > via cosine similarity

In [ ]:
def search_documents(query, n_results=2):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    return results['documents'][0]

# 6) LLM call:-

In [ ]:
def generate_response(query, context):
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {query}

Answer:"""
    
    response = client.models.generate_content(
        model='gemini-2.5-flash-lite',
        contents=prompt
    )
    return response.text

In [ ]:
def rag_query(query):
    relevant_docs = search_documents(query)
    context = "\n".join(relevant_docs)
    answer = generate_response(query, context)
    
    print(f"Question: {query}\n")
    print(f"Context:\n{context}\n")
    print(f"Answer:\n{answer}")
    
    return answer

# examples:-

In [ ]:
rag_query("What is Python?")

In [ ]:
rag_query("How does machine learning work?")

In [ ]:
rag_query("Tell me about React")